In [1]:
from pathlib import Path
from tqdm.notebook import tqdm
import librosa
import librosa.display
import numpy as np
import soundfile
import pandas as pd
import joblib

import random
import os

In [2]:
#PATH_DATASET = Path.cwd()
PATH_DATASET = Path.home() / "datasets/kaggle/birdclef-2021"

df_train_meta = pd.read_csv(PATH_DATASET/"train_metadata.csv")
df_train_meta

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,date,filename,license,rating,time,url
0,acafly,['amegfi'],"['begging call', 'call', 'juvenile']",35.3860,-84.1250,Empidonax virescens,Acadian Flycatcher,Mike Nelson,2012-08-12,XC109605.ogg,Creative Commons Attribution-NonCommercial-Sha...,2.5,09:30,https://www.xeno-canto.org/109605
1,acafly,[],['call'],9.1334,-79.6501,Empidonax virescens,Acadian Flycatcher,Allen T. Chartier,2000-12-26,XC11209.ogg,Creative Commons Attribution-NonCommercial-Sha...,3.0,?,https://www.xeno-canto.org/11209
2,acafly,[],['call'],5.7813,-75.7452,Empidonax virescens,Acadian Flycatcher,Sergio Chaparro-Herrera,2012-01-10,XC127032.ogg,Creative Commons Attribution-NonCommercial-Sha...,3.0,15:20,https://www.xeno-canto.org/127032
3,acafly,['whwbec1'],['call'],4.6717,-75.6283,Empidonax virescens,Acadian Flycatcher,Oscar Humberto Marin-Gomez,2009-06-19,XC129974.ogg,Creative Commons Attribution-NonCommercial-Sha...,3.5,07:50,https://www.xeno-canto.org/129974
4,acafly,['whwbec1'],['call'],4.6717,-75.6283,Empidonax virescens,Acadian Flycatcher,Oscar Humberto Marin-Gomez,2009-06-19,XC129981.ogg,Creative Commons Attribution-NonCommercial-Sha...,3.5,07:50,https://www.xeno-canto.org/129981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62869,yetvir,[],"['adult', 'male', 'song']",30.2150,-97.6505,Vireo flavifrons,Yellow-throated Vireo,Caleb Helsel,2020-07-10,XC591680.ogg,Creative Commons Attribution-NonCommercial-Sha...,1.0,08:30,https://www.xeno-canto.org/591680
62870,yetvir,[],"['life stage uncertain', 'sex uncertain', 'song']",42.3005,-72.5877,Vireo flavifrons,Yellow-throated Vireo,Christopher McPherson,2019-05-31,XC600085.ogg,Creative Commons Attribution-NonCommercial-Sha...,5.0,09:30,https://www.xeno-canto.org/600085
62871,yetvir,"['amered', 'eawpew', 'norcar', 'reevir1']","['adult', 'male', 'song']",42.3005,-72.5877,Vireo flavifrons,Yellow-throated Vireo,Christopher McPherson,2020-06-02,XC602701.ogg,Creative Commons Attribution-NonCommercial-Sha...,4.5,08:30,https://www.xeno-canto.org/602701
62872,yetvir,[],['uncertain'],32.2357,-99.8811,Vireo flavifrons,Yellow-throated Vireo,Brad Banner,2019-04-27,XC614733.ogg,Creative Commons Attribution-NonCommercial-Sha...,4.0,17:30,https://www.xeno-canto.org/614733


In [3]:
cols_taken = [
    "primary_label",
    "latitude",
    "longitude",
    "date",
    "filename",
]

In [4]:
df_meta = df_train_meta.loc[:, cols_taken]
df_meta

,primary_label,latitude,longitude,date,filename
0,acafly,35.3860,-84.1250,2012-08-12,XC109605.ogg
1,acafly,9.1334,-79.6501,2000-12-26,XC11209.ogg
2,acafly,5.7813,-75.7452,2012-01-10,XC127032.ogg
3,acafly,4.6717,-75.6283,2009-06-19,XC129974.ogg
4,acafly,4.6717,-75.6283,2009-06-19,XC129981.ogg
...,...,...,...,...,...
62869,yetvir,30.2150,-97.6505,2020-07-10,XC591680.ogg
62870,yetvir,42.3005,-72.5877,2019-05-31,XC600085.ogg
62871,yetvir,42.3005,-72.5877,2020-06-02,XC602701.ogg
62872,yetvir,32.2357,-99.8811,2019-04-27,XC614733.ogg


We want to parse the column `"date"` into `year, month, day`. What data type is that column?

In [5]:
df_meta.dtypes

primary_label     object
latitude         float64
longitude        float64
date              object
filename          object
dtype: object

In [6]:
type(df_meta.loc[0, "date"])

str

In [7]:
type(df_meta.loc[:, "date"])

pandas.core.series.Series

In [8]:
[s for s in dir(df_meta.loc[:, "date"]) if not s.startswith("_")]

['T',
 'abs',
 'add',
 'add_prefix',
 'add_suffix',
 'agg',
 'aggregate',
 'align',
 'all',
 'any',
 'append',
 'apply',
 'argmax',
 'argmin',
 'argsort',
 'array',
 'asfreq',
 'asof',
 'astype',
 'at',
 'at_time',
 'attrs',
 'autocorr',
 'axes',
 'backfill',
 'between',
 'between_time',
 'bfill',
 'bool',
 'clip',
 'combine',
 'combine_first',
 'compare',
 'convert_dtypes',
 'copy',
 'corr',
 'count',
 'cov',
 'cummax',
 'cummin',
 'cumprod',
 'cumsum',
 'describe',
 'diff',
 'div',
 'divide',
 'divmod',
 'dot',
 'drop',
 'drop_duplicates',
 'droplevel',
 'dropna',
 'dtype',
 'dtypes',
 'duplicated',
 'empty',
 'eq',
 'equals',
 'ewm',
 'expanding',
 'explode',
 'factorize',
 'ffill',
 'fillna',
 'filter',
 'first',
 'first_valid_index',
 'flags',
 'floordiv',
 'ge',
 'get',
 'groupby',
 'gt',
 'hasnans',
 'head',
 'hist',
 'iat',
 'idxmax',
 'idxmin',
 'iloc',
 'index',
 'infer_objects',
 'interpolate',
 'is_monotonic',
 'is_monotonic_decreasing',
 'is_monotonic_increasing',
 'is_uni

In [9]:
# Wrong solution
df_meta.loc[:, "date"].map(lambda s: s.split("-"))[0]

['2012', '08', '12']

In [10]:
df_meta[["year", "month", "day"]] = pd.DataFrame([ date.split("-") for date in df_meta["date"].tolist() ])
df_meta

,primary_label,latitude,longitude,date,filename,year,month,day
0,acafly,35.3860,-84.1250,2012-08-12,XC109605.ogg,2012,08,12
1,acafly,9.1334,-79.6501,2000-12-26,XC11209.ogg,2000,12,26
2,acafly,5.7813,-75.7452,2012-01-10,XC127032.ogg,2012,01,10
3,acafly,4.6717,-75.6283,2009-06-19,XC129974.ogg,2009,06,19
4,acafly,4.6717,-75.6283,2009-06-19,XC129981.ogg,2009,06,19
...,...,...,...,...,...,...,...,...
62869,yetvir,30.2150,-97.6505,2020-07-10,XC591680.ogg,2020,07,10
62870,yetvir,42.3005,-72.5877,2019-05-31,XC600085.ogg,2019,05,31
62871,yetvir,42.3005,-72.5877,2020-06-02,XC602701.ogg,2020,06,02
62872,yetvir,32.2357,-99.8811,2019-04-27,XC614733.ogg,2019,04,27


## Next Up: Cut The Audio Files
- All cut audio files will serve as training data
- **Not really cut**, but read the audio files and then take slices of 5 sec to save into `.npy` files

In [11]:
PATH_DATASET.name

'birdclef-2021'

In [12]:
#L_short_ogg_paths = list((PATH_DATASET / "train_short_audio").glob("*/*.ogg"))
#L_short_ogg_paths[:10]
#len(L_short_ogg_paths)

In [13]:
L_ogg_paths = (PATH_DATASET / "train_short_audio").glob("*/*.ogg")

In [14]:
save_to = Path("short_audio")
save_to.exists()

True

In [15]:
save_to = Path("short_audio")
save_to.mkdir(exist_ok=True)
save_to.exists()

True

In [22]:
from utils import *

The above cells took really long on most computers (circa one hour or one and a half). Maybe there are ways to improve it -- `async` or sth.

### Train/Val Sets
Since the npy data are too big to fit in RAM, we will

- separate them into train and validation sets (i.e. real directories for the two sets)
- try to use `keras.preprocessing.image.ImageDataGenerator` to create train/val generators

First we construct a dataframe containing the information on the cut `.npy` files


In [27]:
L_npy_filenames = [path.name for path in save_to.glob("*.npy")]
L_npy_filenames[:20]

['XC11209_5.npy',
 'XC11209_10.npy',
 'XC11209_15.npy',
 'XC129974_5.npy',
 'XC129974_10.npy',
 'XC127032_5.npy',
 'XC109605_5.npy',
 'XC127032_10.npy',
 'XC129981_5.npy',
 'XC127032_15.npy',
 'XC109605_10.npy',
 'XC129981_10.npy',
 'XC127032_20.npy',
 'XC109605_15.npy',
 'XC127032_25.npy',
 'XC129981_15.npy',
 'XC130056_5.npy',
 'XC109605_20.npy',
 'XC127032_30.npy',
 'XC130056_10.npy']

In [30]:
df_meta.head()

,primary_label,latitude,longitude,date,filename,year,month,day
0,acafly,35.3860,-84.1250,2012-08-12,XC109605.ogg,2012,08,12
1,acafly,9.1334,-79.6501,2000-12-26,XC11209.ogg,2000,12,26
2,acafly,5.7813,-75.7452,2012-01-10,XC127032.ogg,2012,01,10
3,acafly,4.6717,-75.6283,2009-06-19,XC129974.ogg,2009,06,19
4,acafly,4.6717,-75.6283,2009-06-19,XC129981.ogg,2009,06,19


In [32]:
df_xc = pd.DataFrame(np.c_[
    [filename.split("_")[0] + ".ogg" for filename in L_npy_filenames],
     L_npy_filenames,
    ],
    columns=["filename", "npy_filename"]
)
df_xc

,filename,npy_filename
0,XC11209.ogg,XC11209_5.npy
1,XC11209.ogg,XC11209_10.npy
2,XC11209.ogg,XC11209_15.npy
3,XC129974.ogg,XC129974_5.npy
4,XC129974.ogg,XC129974_10.npy
...,...,...
676621,XC602701.ogg,XC602701_80.npy
676622,XC602701.ogg,XC602701_85.npy
676623,XC602701.ogg,XC602701_90.npy
676624,XC602701.ogg,XC602701_95.npy


Next we want to **join** the two dataframes `df_meta` and `df_xc`.

In [43]:
df_join = df_meta.join(df_xc.set_index("filename"), on="filename")
#df_join = df_meta.join(df_xc, on="filename")  # Wrong!
#df_join = df_meta.join(df_xc, rsuffix="_right")  # Wrong!
df_join

,primary_label,latitude,longitude,date,filename,year,month,day,npy_filename
0,acafly,35.386,-84.1250,2012-08-12,XC109605.ogg,2012,08,12,XC109605_5.npy
0,acafly,35.386,-84.1250,2012-08-12,XC109605.ogg,2012,08,12,XC109605_10.npy
0,acafly,35.386,-84.1250,2012-08-12,XC109605.ogg,2012,08,12,XC109605_15.npy
0,acafly,35.386,-84.1250,2012-08-12,XC109605.ogg,2012,08,12,XC109605_20.npy
0,acafly,35.386,-84.1250,2012-08-12,XC109605.ogg,2012,08,12,XC109605_25.npy
...,...,...,...,...,...,...,...,...,...
62873,yetvir,31.906,-109.1543,2020-05-26,XC615888.ogg,2020,05,26,XC615888_55.npy
62873,yetvir,31.906,-109.1543,2020-05-26,XC615888.ogg,2020,05,26,XC615888_60.npy
62873,yetvir,31.906,-109.1543,2020-05-26,XC615888.ogg,2020,05,26,XC615888_65.npy
62873,yetvir,31.906,-109.1543,2020-05-26,XC615888.ogg,2020,05,26,XC615888_70.npy


In [44]:
len(df_join)

676626

In [38]:
df_join.index

Int64Index([    0,     0,     0,     0,     0,     0,     0,     0,     0,
                0,
            ...
            62873, 62873, 62873, 62873, 62873, 62873, 62873, 62873, 62873,
            62873],
           dtype='int64', length=676626)

In [47]:
df_join.reset_index(drop=True, inplace=True)

In [48]:
df_join.head()

,primary_label,latitude,longitude,date,filename,year,month,day,npy_filename
0,acafly,35.386,-84.125,2012-08-12,XC109605.ogg,2012,08,12,XC109605_5.npy
1,acafly,35.386,-84.125,2012-08-12,XC109605.ogg,2012,08,12,XC109605_10.npy
2,acafly,35.386,-84.125,2012-08-12,XC109605.ogg,2012,08,12,XC109605_15.npy
3,acafly,35.386,-84.125,2012-08-12,XC109605.ogg,2012,08,12,XC109605_20.npy
4,acafly,35.386,-84.125,2012-08-12,XC109605.ogg,2012,08,12,XC109605_25.npy


In [49]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=SEED)
for train_indices, test_indices in split.split(df_join, df_join["primary_label"]):
    df_train = df_join.loc[train_indices]
    df_test = df_join.loc[test_indices]

We can verify our splits like this:

In [52]:
df_train["primary_label"].value_counts()

cubthr     7332
eursta     6102
normoc     6095
bncfly     4859
amerob     4770
           ... 
hofwoo1      53
runwre1      42
whcpar       36
wegspa1      33
stvhum2      29
Name: primary_label, Length: 397, dtype: int64

In [54]:
df_train["primary_label"].value_counts() / df_join["primary_label"].value_counts()

acafly     0.700000
acowoo     0.700183
aldfly     0.700048
ameavo     0.699060
amecro     0.700000
             ...   
yelwar     0.699885
yeofly1    0.700087
yerwar     0.699950
yeteup1    0.701099
yetvir     0.700203
Name: primary_label, Length: 397, dtype: float64

In [55]:
df_test["primary_label"].value_counts() / df_join["primary_label"].value_counts()

acafly     0.300000
acowoo     0.299817
aldfly     0.299952
ameavo     0.300940
amecro     0.300000
             ...   
yelwar     0.300115
yeofly1    0.299913
yerwar     0.300050
yeteup1    0.298901
yetvir     0.299797
Name: primary_label, Length: 397, dtype: float64

## Parameters
Let's introduce a few convenience variables.

In [ ]:
SR = 32_000
DURATION = 5
SEED = 42

In [ ]:
PATH_DATASET = Path.home() / "datasets/kaggle/birdclef-2021"
PATH_DATASET.exists()

In [ ]:
L_birds = [path.name for path
           in (PATH_DATASET / "train_short_audio").iterdir()]
L_birds[:10]

In [ ]:
L_sorted = sorted(L_birds)
if L_birds != L_sorted:
    print("False")
    L_birds = L_sorted

In [ ]:
D_label_index = {label: i for i, label in enumerate(L_birds)}
D_label_index

In [ ]:
D_index_label = {v: k for k, v in D_label_index.items()}
D_index_label

## Melspectrogram
Let's study how to cut the audio files into pieces of 5 sec each and transform them into melspectrogram.

In [ ]:
class MelSpecComputer:
    def __init__(self, sr, n_mels, fmin, fmax, **kwargs):
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax
        kwargs["n_fft"] = kwargs.get("n_fft", self.sr//10)
        kwargs["hop_length"] = kwargs.get("hop_length", self.sr//(10*4))
        self.kwargs = kwargs

    def __call__(self, y):
        melspec = librosa.feature.melspectrogram(
            y,
            sr=self.sr,
            n_mels=self.n_mels,
            fmin=self.fmin,
            fmax=self.fmax,
            **self.kwargs,
        )

        melspec = librosa.power_to_db(melspec).astype(np.float32)
        return melspec

In [ ]:
def standardize_uint8(X, eps=1e-6, mean=None, std=None):
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)
    
    min_, max_ = X.min(), X.max()
    if max_ - min_ > eps:
        #V = np.clip(X, min_, max_)
        #V = 255 * (V - min_) / (max_ - min_)
        V = 255 * (X - min_) / (max_ - min_)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(X, dtype=np.uint8)
    return V

def crop_or_pad(y, length, is_train=True, start=None):
    """
    crop or pad the signal y to #(samples) = `length`
      - repetition of itself
      - random truncating
    """
    if len(y) < length:
        #y = np.concatenate([y, np.zeros(length - len(y))])
        n_repeats = length // len(y)
        remainder = length % len(y)
        y = np.concatenate([y]*n_repeats + [y[:remainder]])
    elif len(y) > length:
        if not is_train:
            start = start or 0
        else:
            start = start or np.random.randint(len(y) - length)
        y = y[start:start + length]
    return y

In [ ]:
path_random_short_audio = random.choice(
    list((PATH_DATASET / "train_short_audio").glob("*/*.ogg"))
)
print(f"path_random_short_audio =\n{path_random_short_audio}")


# Excerpt from `class AudioToImage`
audio, orig_sr = soundfile.read(path_random_short_audio, dtype="float32")

if orig_sr != SR:
    audio = librosa.resample(audio, orig_sr, SR, res_type="kaiser_fast")

audio = crop_or_pad(audio, length=SR*DURATION)
mel_spec_computer = MelSpecComputer(
    sr=SR,
    n_mels=128,
    fmin=0,
    fmax=None,
)
melspec = mel_spec_computer(audio) 
image = standardize_uint8(melspec)
type(image), image.shape

In [ ]:
librosa.display.specshow(image);

These melspectrograms seem to be all single-channeled. Besides, probably due to the fact we have always taken the same number of samples (after resampling with a common `SR`), we always obtain an ndarray of shape `(128, 201)`.

In [ ]:
list(range(0, 88, 5))

## Statistics on `train_short_audio/`
I would like to know some statistics like

- How many `.ogg` files does each bird label have?
  - Linux's `tree` command might be able to answer to this question.
- How long are these `.ogg` files? Are some a lot longer than others?

In [ ]:
!tree $PATH_DATASET/train_short_audio/* --filelimit 3

Well, if we are to use `tree`, we'd be better off writing a whole shell script. Let's try it Python way.

In [ ]:
PATH_DATASET.stat()

In [ ]:
PATH_DATASET.stat().st_size

In [ ]:
len(list(PATH_DATASET.iterdir()))

`st_size` is the size of the file, i.e. how many bytes.

In [ ]:
!ls -ld $PATH_DATASET

In [ ]:
D_label_nShort = dict()

In [ ]:
for p in (PATH_DATASET / "train_short_audio").iterdir():
    n_oggs = len(set(p.iterdir()))
    print(f"{p.name:12s}{n_oggs: 5d}")
    D_label_nShort[p.name] = n_oggs

In [ ]:
def duration(path_ogg):
    audio, orig_sr = soundfile.read(path_ogg, dtype="float32")
    audio = librosa.resample(audio, orig_sr, SR, res_type="kaiser_fast")
    return len(audio) / SR

The following cell can display the number of seconds each `.ogg` file lasts, but it's **time-consuming**; we'd better combine it with other loops for reading all the short audio files.
> But the result show that the duration is not quite homogeneous: it could range from as low as 6 seconds to sth like 200 seconds.

## `train_soundscapes/`

In [ ]:
df_train_soundscape = pd.read_csv(PATH_DATASET/"train_soundscape_labels.csv")
df_train_soundscape.head()

In [ ]:
df_train_soundscape.notnull().all()

There are a total of `20` `.ogg` files in `train_soundscapes/`: I would like to split these into train/val/test sets.

- `12` files for train
- `4` files for val
- `4` files for test

### Nota Bene
Just be careful: We decided to split the training data given by Kaggle into 3 parts -- train/val/test. So the word **`train`** in what follows may

- either means `train-train`
- or just `train`

Sorry for the confusion and the abuse, but unless I find time or I wouldn't correct it.

In [ ]:
n_train = 12
n_val = 4     # Maybe I will put train and val together
n_test = 4

In [ ]:
n_test, n_train, n_val

In [ ]:
test_size = n_test / sum([n_test, n_train, n_val])
test_size

In [ ]:
L_paths_train_soundscape     = sorted(list((PATH_DATASET / "train_soundscapes").iterdir()))
L_paths_train_soundscape_SSW = sorted(list((PATH_DATASET / "train_soundscapes").glob("*SSW*.ogg")))
L_paths_train_soundscape_COR = sorted(list((PATH_DATASET / "train_soundscapes").glob("*COR*.ogg")))
[ p.name for p in L_paths_train_soundscape ]

In [ ]:
## Old deprecated code -- because of non-reproducibility
#L_testSoundScapeIDs = [int(p.name.split("_")[0]) for p in L_paths_train_soundscape_SSW[-n_test//2:]]
#L_testSoundScapeIDs.extend([int(p.name.split("_")[0]) for p in L_paths_train_soundscape_COR[-n_test//2:]])
#S_testSoundScapeIDs = set(L_testSoundScapeIDs)
#L_testSoundScapeIDs

In [ ]:
from sklearn.model_selection import train_test_split

L_SSW_IDs = [int(p.name.split("_")[0]) for p in L_paths_train_soundscape_SSW]
L_COR_IDs = [int(p.name.split("_")[0]) for p in L_paths_train_soundscape_COR]

L_trainSoundScape_SSW_IDs, L_testSoundScape_SSW_IDs, L_trainSoundScape_COR_IDs, L_testSoundScape_COR_IDs = train_test_split(
    L_SSW_IDs, L_COR_IDs, test_size=test_size, random_state=SEED)

print(f"L_trainSoundScape_SSW_IDs = {L_trainSoundScape_SSW_IDs}")
print(f"L_testSoundScape_SSW_IDs = {L_testSoundScape_SSW_IDs}")
print(f"L_trainSoundScape_COR_IDs = {L_trainSoundScape_COR_IDs}")
print(f"L_testSoundScape_COR_IDs = {L_testSoundScape_COR_IDs}")

In [ ]:
L_trainSoundScapeIDs = L_trainSoundScape_SSW_IDs + L_trainSoundScape_COR_IDs
L_testSoundScapeIDs = L_testSoundScape_SSW_IDs + L_testSoundScape_COR_IDs
S_trainSoundScapeIDs = set(L_trainSoundScapeIDs)
S_testSoundScapeIDs = set(L_testSoundScapeIDs)
print(f"S_trainSoundScapeIDs = {S_trainSoundScapeIDs}")
print(f"S_testSoundScapeIDs = {S_testSoundScapeIDs}")

In [ ]:
df_train_soundscape.columns

In [ ]:
df_train_soundscape.head()

In [ ]:
PATH_DATASET._str

In [ ]:
df_train_soundscape.dtypes

In [ ]:
df_train_soundscape.audio_id

In [ ]:
df_train_soundscape.audio_id.unique()

In [ ]:
df_train_soundscape.audio_id.any() in L_testSoundScapeIDs

In [ ]:
L_testSoundScapeIDs

In [ ]:
L_testSoundScapeIDs[0] in df_train_soundscape.audio_id

In [ ]:
L_testSoundScapeIDs[0] in df_train_soundscape.audio_id.unique()

In [ ]:
df_train_soundscape["is_test"] = [ (id_ in S_testSoundScapeIDs)
                                  for id_ in df_train_soundscape.audio_id]
df_train_soundscape.head()

`mkdir` for places to store melspectrogram `.npy` files

In [ ]:
trainSoundScapes = Path("./trainSoundScapes")
trainSoundScapes.exists()

In [ ]:
trainSoundScapes.mkdir(exist_ok=True)
trainSoundScapes.exists()

In [ ]:
testSoundScapes = Path("./testSoundScapes")
testSoundScapes.exists()

In [ ]:
testSoundScapes.mkdir(exist_ok=True)
testSoundScapes.exists()

In [ ]:
ls

`NamedTuple` and `itertuples()` method from dataframe

In [ ]:
list((PATH_DATASET / "train_soundscapes").glob("*7019_COR*.ogg"))

In [ ]:
next((PATH_DATASET / "train_soundscapes").glob("*7019_COR*.ogg"))

In [ ]:
class AudioToImage:
    def __init__(self,
                 sr=SR,
                 n_mels=128,
                 fmin=0,
                 fmax=None,
                 duration=DURATION,
                 step=None,
                 res_type="kaiser_fast",
                 resample=True):
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax or self.sr // 2

        self.duration = duration
        self.audio_length = self.duration*self.sr  # this is the #(samples) corresponding to DURATION=5 sec
        self.step = step or self.audio_length
        
        self.res_type = res_type
        self.resample = resample

        self.mel_spec_computer = MelSpecComputer(sr=self.sr,
                                                 n_mels=self.n_mels,
                                                 fmin=self.fmin,
                                                 fmax=self.fmax)
        
    def audio_to_image(self, audio):
        melspec = self.mel_spec_computer(audio) 
        image = standardize_uint8(melspec)
        return image

    def __call__(self, row, save=True, save_to=None, is_short_audio=False):
        #max_audio_duration = 10*self.duration
        #init_audio_length = max_audio_duration*row.sr
        #start = 0 if row.duration <  max_audio_duration else np.random.randint(row.frames - init_audio_length)

        if save and save_to is None:
            print("Nothing was done. If you want to save=True, please specify save_to=some_path")
            return
    
        #audio, orig_sr = soundfile.read(row.filepath, dtype="float32")

        #if self.resample and orig_sr != self.sr:
        #    audio = librosa.resample(audio, orig_sr, self.sr, res_type=self.res_type)
          
        #audios = [audio[i:i+self.audio_length] for i in range(0, max(1, len(audio) - self.audio_length + 1), self.step)]
        #audios[-1] = crop_or_pad(audios[-1] , length=self.audio_length)
        #images = [self.audio_to_image(audio) for audio in audios]
        #images = np.stack(images)
        if not is_short_audio:
            # Then audio from train_soundscapes/
            # an audio every 5 sec
            path_ogg = next((PATH_DATASET / "train_soundscapes").glob(f"*{row.audio_id}_{row.site}*.ogg"))
            audio, orig_sr = soundfile.read(path_ogg, dtype="float32")
            if self.resample and orig_sr != self.sr:
                audio = librosa.resample(audio, orig_sr, self.sr, res_type=self.res_type)
            start_index = (row.seconds - self.duration) * self.sr
            audio = audio[start_index:start_index + self.audio_length]
            image = self.audio_to_image(audio)
          
        if save:
            #path = TRAIN_AUDIO_IMAGES_SAVE_ROOT/f"{row.primary_label}/{row.filename}.npy"
            path = save_to / f"{row.row_id}.npy"
            path.parent.mkdir(exist_ok=True, parents=True)
            np.save(str(path), image)
        else:
            return  row.row_id, image


def get_audios_as_images(df, save_to=None):
    pool = joblib.Parallel(4)
    
    #converter = AudioToImage(step=int(DURATION*0.666*SR))
    #def __call__(self, row, save=True, save_to=None, is_short_audio=False):
    converter = AudioToImage()
    mapper = joblib.delayed(converter)
    #tasks = [mapper(row) for row in df.itertuples(False)]
    tasks = [mapper(row, save_to=save_to) for row in df.itertuples(False)]
    
    pool(tqdm(tasks))

### So Wrong
There are a few drawbacks in the previous two cells:
- It was wrong by saving all `.npy` files to `trainSoundScape` directory: Some should be saved to `testSoundScape` directory.
- It is not sufficient because the same `.ogg` file is opened and read several times.
  - Write a function or a class of your own and stop using kkiller's in this case.

### Objectives: Save `.npy` Files
Let's make our objectives clear.

01. We want to save each `train_soundscapes/` audio file, each 5-sec clip, to an `.npy` file.
  - The file reading and melspectrogram transform might be quite blocking, so **asynchronous** or **multi-process** (e.g. `concurrent.futures.ProcessPoolExecutor` or `joblib`) programming might be useful here.
02. We want to update `df_train_soundscape` to contain more information. What information?
  - Date: Can be separated.
  - Corresponding `.npy` path: Can be separated.
  - Longitude, latitude: Can be separated.
  
Since the two objectives above can be separated, and updating an extra dataframe while saving `.npy` for each `.ogg` file can become quite complicated, let's first get objective 1 done and start objective 2 after that.

### Objective 1: `.ogg` to `.npy`

#### `joblib` way

In [ ]:
list(range(10))[0:-3]

In [ ]:
list(range(10))[:-3]

In [ ]:
## deprecated code
#S_trainSoundScapeIDs = set(int(p.name.split("_")[0]) for p in L_paths_train_soundscape_SSW[:-n_test//2])
#S_trainSoundScapeIDs.update(set(int(p.name.split("_")[0]) for p in L_paths_train_soundscape_COR[:-n_test//2]))
#S_trainSoundScapeIDs#, len(S_trainSoundScapeIDs)

In [ ]:
next((PATH_DATASET / "train_soundscapes").glob(f"50878*.ogg"))

In [ ]:
np.arange(10)[9:100]

In [ ]:
id_ = 1391
location = "SWC"
n_samples_5sec = 231
i = 6 * n_samples_5sec
f"{id_}_{location}_{((i + n_samples_5sec) // n_samples_5sec) * 5}.npy"

In [ ]:
def audio_to_mels(audio,
                  sr=SR,
                  n_mels=128,
                  fmin=0,
                  fmax=None):
    fmax = fmax or sr // 2
    mel_spec_computer = MelSpecComputer(sr=sr,
                                        n_mels=n_mels,
                                        fmin=fmin,
                                        fmax=fmax)
    mels = standardize_uint8(mel_spec_computer(audio))
    return mels

def every_5sec(id_,
               sr=SR,
               resample=True,
               res_type="kaiser_fast",
               single_process=True,
               save_to=Path("corbeille"),
               n_workers=2
                ):
    """
    - read the audio file of ID `id_`
    - cut the read audio into pieces of 5 seconds
    - convert each piece into `.npy` file and save
    """
    path_ogg = next((PATH_DATASET / "train_soundscapes").glob(f"{id_}*.ogg"))
    location = (path_ogg.name).split("_")[1]
    whole_audio, orig_sr = soundfile.read(path_ogg, dtype="float32")
    if resample and orig_sr != sr:
        whole_audio = librosa.resample(whole_audio, orig_sr, sr, res_type=res_type)
    n_samples = len(whole_audio)
    n_samples_5sec = sr * 5
    save_to.mkdir(exist_ok=True)

    def convert_and_save(i):
        audio_i = whole_audio[i:i + n_samples_5sec]
        mels_i = audio_to_mels(audio_i)
        path_i = save_to / f"{id_}_{location}_{((i + n_samples_5sec) // n_samples_5sec) * 5}.npy"
        np.save(str(path_i), mels_i)

    if single_process:
        for i in range(0, n_samples - n_samples % n_samples_5sec, n_samples_5sec):
            #audio_i = whole_audio[i:i + n_samples_5sec]
            ## No need the next check because in range() we have subtracted the remainder.
            ## That is, len(audio_i) is guaranteed to be n_samples_5sec for all i.
            ##if len(audio_i) < n_samples_5sec:
            ##    pass
            #mels_i = audio_to_mels(audio_i)
            #path_i = save_to / f"{id_}_{location}_{((i + n_samples_5sec) // n_samples_5sec) * 5}.npy"
            #np.save(str(path_i), mels_i)
            convert_and_save(i)
    else:
        pool = joblib.Parallel(n_workers)
        mapping = joblib.delayed(convert_and_save)
        tasks = (mapping(i) for i in range(0, n_samples - n_samples % n_samples_5sec, n_samples_5sec))
        pool(tasks)

def soundscapes_to_npy(is_test=False, n_processes=4):
    pool = joblib.Parallel(n_processes)
    mapping = joblib.delayed(every_5sec)
    if is_test:
        tasks = list(mapping(id_, save_to=testSoundScapes) for id_ in S_testSoundScapeIDs)
        #tasks = list(mapping(id_,
        #                     single_process=False,
        #                     save_to=testSoundScapes)
        #             for id_ in S_testSoundScapeIDs)
    else:
        tasks = list(mapping(id_, save_to=trainSoundScapes) for id_ in S_trainSoundScapeIDs)
        #tasks = list(mapping(id_,
        #                     single_process=False,
        #                     save_to=trainSoundScapes)
        #             for id_ in S_trainSoundScapeIDs)
    pool(tqdm(tasks))

### Nota Bene
- `tasks` (i.e. input to `joblib.Parallel`) can be either a generator or a list, but since I do not know a priori the length of a generator, when combined with the usage of `tqdm`, the progress bar will lack the capability to show progress percentage, compared to using a list.

In [ ]:
%%time
soundscapes_to_npy()

In [ ]:
soundscapes_to_npy(is_test=True)

In [ ]:
S_testSoundScapeIDs

In [ ]:
!ls $trainSoundScapes | wc -l

In [ ]:
!ls $testSoundScapes | wc -l

In [ ]:
16 * (600 // 5)

In [ ]:
4 * (600 // 5)

Let's at least verify that the saved images exhibit difference.<br>
Try execute the next cell several times to see randomly the melspectrograms.

In [ ]:
rand_npy = random.choice(list(trainSoundScapes.iterdir()))
rand_image = np.load(rand_npy)
print(f"rand_npy = {rand_npy.name}")
librosa.display.specshow(rand_image);

### Objective 2: Construct `df_train_soundscape`

Recall that
> - We want to update `df_train_soundscape` to contain more information. What information?
>   - Date: Can be separated.
>   - Corresponding `.npy` path: Can be separated.
>   - Longitude, latitude: Can be separated.
>   - birds label to birds indices?
>   - new col `"n_birds"` and do a stat?

Construct a dictionary for

- key: recording location, e.g. `COR`, `SSW`, etc.
- value: possibly `NamedTuple(longitude, latitude)`

In [ ]:
from collections import namedtuple
Coordinate = namedtuple("Coordinate", ["longitude", "latitude"])

D_location_coordinate = dict()
for p in (PATH_DATASET / "test_soundscapes").glob("*_recording_location.txt"):
    location = p.stem.split("_")[0]
    with open(p) as f:
        lines = f.readlines()
        for line in lines:
            if line.startswith("Latitude:"):
                latitude = float(line.split(" ")[1])
            if line.startswith("Longitude:"):
                longitude = float(line.split(" ")[1])
    D_location_coordinate[location] = Coordinate(longitude=longitude, latitude=latitude)
D_location_coordinate

I think the year won't make much difference.

In [ ]:
rand_npy

In [ ]:
rand_npy.stem

In [ ]:
rand_npy.stem.split("_")

In [ ]:
# Randomly initialize the new columns
df_train_soundscape["year"] = -1
df_train_soundscape["month"] = -1
df_train_soundscape["day"] = -1
df_train_soundscape["longitude"] = 365.0
df_train_soundscape["latitude"] = 365.0
#df_train_soundscape["npy_path"] = None
df_train_soundscape.head()

In [ ]:
df_train_soundscape["year"].dtype

In [ ]:
df_train_soundscape["longitude"].dtype

In [ ]:
df_train_soundscape["row_id"][df_train_soundscape.audio_id == 7019]

In [ ]:
for p in (PATH_DATASET / "train_soundscapes").iterdir():
    id_, location, date = p.stem.split("_")
    # date, str, "yyyymmdd"
    year = int(date[:4])
    month = int(date[4:6])
    day = int(date[6:])
    id_ = int(id_)
    filter_ = df_train_soundscape.audio_id == id_
    #df_train_soundscape.loc[filter_, ["year"]] = year
    #df_train_soundscape.loc[filter_, ["month"]] = month
    #df_train_soundscape.loc[filter_, ["day"]] = day
    df_train_soundscape.loc[filter_, "year"] = year
    df_train_soundscape.loc[filter_, "month"] = month
    df_train_soundscape.loc[filter_, "day"] = day
    #is_test = id_ in S_testSoundScapeIDs
df_train_soundscape

We use string concatenation for the construction of the column `"npy_path"`.

In [ ]:
str(PATH_DATASET)

In [ ]:
df_train_soundscape["npy_parent"] = ""
is_test = df_train_soundscape.is_test == True
is_train = ~is_test
df_train_soundscape.loc[is_test, "npy_parent"] = str(testSoundScapes)
df_train_soundscape.loc[is_train, "npy_parent"] = str(trainSoundScapes)
df_train_soundscape.loc[is_train].head()

In [ ]:
df_train_soundscape.loc[is_test].head()

In [ ]:
df_train_soundscape["npy_path"] = \
    df_train_soundscape["npy_parent"] + "/" + \
    df_train_soundscape["row_id"] + ".npy"

# or equiv.
#df_train_soundscape.loc[:, "npy_path"] = \
#    df_train_soundscape.loc[:, "npy_parent"] + "/" + \
#    df_train_soundscape.loc[:, "row_id"] + ".npy"

In [ ]:
df_train_soundscape.loc[is_test, ["row_id", "npy_path"]]

In [ ]:
df_train_soundscape.loc[is_train, ["row_id", "npy_path"]]

For the columns `"longitude", "latitude"`, we will loop thru `D_location_coordinate`.

In [ ]:
D_location_coordinate

In [ ]:
lo, la = D_location_coordinate["COR"]
lo, la

In [ ]:
for location, coordinate in D_location_coordinate.items():
    lo, la= coordinate.longitude, coordinate.latitude
    location_filter = df_train_soundscape.loc[:, "site"] == location
    df_train_soundscape.loc[location_filter, "longitude"] = lo
    df_train_soundscape.loc[location_filter, "latitude"] = la
df_train_soundscape.loc[:, ["site", "longitude", "latitude"]]

In [ ]:
df_train_soundscape.head()

In [ ]:
type(df_train_soundscape.loc[:, "birds"])

In [ ]:
df_train_soundscape.loc[df_train_soundscape["birds"] != "nocall", "birds"].head(20)

In [ ]:
df_train_soundscape.loc[:, "birds"].values

In [ ]:
I = np.eye(len(D_label_index))

In [ ]:
I.dtype, I.shape

In [ ]:
np.zeros((3,5)).dtype

In [ ]:
I[[1,3,5]]

In [ ]:
np.sum(I[[1,3,5]], axis=0)

In [ ]:
np.sum(I[[1,3,5]], axis=0).shape

In [ ]:
I[3].shape

In [ ]:
def birds_to_ndarry(series):
    I = np.eye(len(D_label_index))
    ndarray = np.zeros((len(series), len(D_label_index)))
    for i, string in enumerate(series.values):
        if string == "nocall":
            continue
        else:
            L_indices = [D_label_index[label] for label in string.split(" ")]
            row_i = np.sum(I[L_indices], axis=0)
            ndarray[i] = row_i
    return ndarray

In [ ]:
A = birds_to_ndarry(df_train_soundscape.birds)
A.shape

In [ ]:
y_train = A[df_train_soundscape.is_test==False]
y_train.shape

In [ ]:
y_test = A[df_train_soundscape.is_test==True]
y_test.shape

In [ ]:
df_train_soundscape

In [ ]:
#df_train_soundscape.to_csv("rich_train_soundscape_labels.csv", index=True)
df_train_soundscape.to_csv("rich_train_soundscape_labels.csv", index=False)

## `train_short_audio/`


In [ ]:
S_trainSoundScapeIDs

In [ ]:
S_testSoundScapeIDs